In [1]:
from rosemary import jpt_setup; jpt_setup()

/dccstor/mit_fm/wpq/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [4]:
import pandas as pd

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem usage (GB)', 'per-epoch time (hr)', 'per-iter time (s)']

# 55k data points, batch_size=128
data_oasst1 = [
    ('gpt2', 0.124, 'bf16', 'no', 10, None, None),
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 2.5, 11),
    # incorporate deep speed is costly!
    ('gpt2-Large', 0.774, 'bf16', 'stage 3 no offloading', 40, 6, 25),
    # 1 a100_40g: without offloading OOM on `.backward()`, runs fine with offloading.
    ('gpt2-xl', 1.5, 'bf16', 'stage 3 with offloading', 40, 13, 55),
    # 4 v100_32g: without offloading.
]

df_oasst1 = pd.DataFrame(data_oasst1, columns=cols)

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem (GB)', 'cpu mem (GB)', 'per-epoch time (hr)', 'per-iter time (s)']
data = [
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 7, 9.5, 9),
]

print('instruction tune human-mix on 1 a100_40g:')
df = pd.DataFrame(data, columns=cols)
df

instruction tune human-mix on 1 a100_40g:


,model,size,mixed-precision,deepspeed,gpu mem (GB),cpu mem (GB),per-epoch time (hr),per-iter time (s)
0,gpt2-Large,0.774,bf16,no,36,7,9.5,9


# Finetuning with openinstruct/finetune.py


In [41]:
# model_name_or_path = 'mosaicml/mpt-7b'; max_seq_length = 2048
model_name_or_path = 'gpt2'; max_seq_length = 1024
model_name_or_path = 'gpt2-Large'; max_seq_length = 1024
# model_name_or_path = 'gpt2-xl'; max_seq_length = 1024


train_file = 'data/processed/oasst1/oasst1_data.jsonl'; train_file_short = 'oasst1'
train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
train_file = 'data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl'; train_file_short = 'human_mix_shuffled'

# output_dir = 'results/mpt-7b_oasst1'
output_dir = f"results/{model_name_or_path.split('/')[-1]}_{train_file_short}"

# deepspeed_config_file = 'ds_configs/stage3_no_offloading_accelerate_setauto.conf'
# deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate.conf'
deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate_setauto.conf'

num_gpus = 1
batch_size_per_gpu = 2
total_batch_size = 128
gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{batch_size_per_gpu} batch size per GPU, "
      f"{gradient_acc_steps} gradient accumulation steps.")

# do use fast tokenizer since mpt-7b does not have a fast tokenizer counter-part
#     --use_slow_tokenizer \
# do not use flash attention, since having problem installing flash-attn with cuda 12.1
#     --use_flash_attn \

#     --use_deepspeed \
#     --deepspeed_config_file {deepspeed_config_file} \


cmd = f"""
!cd .. && \
accelerate launch \
    --mixed_precision bf16 \
    --num_machines 1 \
    --num_processes {num_gpus} \
    open_instruct/finetune.py \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    --train_file {train_file} \
    --max_seq_length {max_seq_length} \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size {batch_size_per_gpu} \
    --gradient_accumulation_steps {gradient_acc_steps} \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    --num_train_epochs 2 \
    --output_dir {output_dir} \
    --with_tracking \
    --report_to tensorboard \
    --logging_steps 1
"""
print(cmd)


Training gpt2-Large using 1 GPUs, 2 batch size per GPU, 64 gradient accumulation steps.

!cd .. && accelerate launch     --mixed_precision bf16     --num_machines 1     --num_processes 1     open_instruct/finetune.py     --model_name_or_path gpt2-Large     --tokenizer_name gpt2-Large     --train_file data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl     --max_seq_length 1024     --preprocessing_num_workers 16     --per_device_train_batch_size 2     --gradient_accumulation_steps 64     --learning_rate 2e-5     --lr_scheduler_type linear     --warmup_ratio 0.03     --weight_decay 0.     --num_train_epochs 2     --output_dir results/gpt2-Large_human_mix_shuffled     --with_tracking     --report_to tensorboard     --logging_steps 1



In [36]:
!cd .. && accelerate launch     --mixed_precision bf16     --num_machines 1     --num_processes 1     open_instruct/finetune.py     --model_name_or_path gpt2-Large     --tokenizer_name gpt2-Large     --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl     --max_seq_length 1024     --preprocessing_num_workers 16     --per_device_train_batch_size 2     --gradient_accumulation_steps 64     --learning_rate 2e-5     --lr_scheduler_type linear     --warmup_ratio 0.03     --weight_decay 0.     --num_train_epochs 2     --output_dir results/gpt2-Large_oasst1     --with_tracking     --report_to tensorboard     --logging_steps 1


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dccstor/mit_fm/miniconda/envs/open-instruct did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/opt/ibm/lsfsuite/ext/ppm/10.2/linux2.6-glibc2.3-x86_64/lib')}
  warn(msg)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 24.46it/s]
loading configuration file config.json from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--gpt2-Large/snapshots/97935fc1a406f447320c3db70fe9e9875dca2595/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-Large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "t

07/06/2023 18:44:35 - INFO - __main__ - ***** Running training *****
07/06/2023 18:44:35 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) =  Num examples = 260960
07/06/2023 18:44:35 - INFO - __main__ -   Num Epochs = 2
07/06/2023 18:44:35 - INFO - __main__ -   Instantaneous batch size per device = 2
07/06/2023 18:44:35 - INFO - __main__ -  128
07/06/2023 18:44:35 - INFO - __main__ -   Gradient Accumulation steps = 64
07/06/2023 18:44:35 - INFO - __main__ -   Total optimization steps = 4078
  2%|▊                                      | 85/4078 [12:00<9:23:19,  8.46s/it]07/06/2023 18:56:35 - INFO - __main__ -   Step: 85, LR: 1.3934426229508198e-05, Loss: 2.152805805206299


  4%|█▌                                    | 173/4078 [24:10<9:05:20,  8.38s/it]07/06/2023 19:08:46 - INFO - __main__ -   Step: 173, LR: 1.9742163801820024e-05, Loss: 1.9252840280532837


  6%|██▍                                   | 261/4078 [36:27<8:44:27,  8.24s/it]07/06/2023 19:21:02 - INFO - __main__ -   Step: 261, LR: 1.929726996966633e-05, Loss: 1.9786278009414673


  7%|██▊                                   | 299/4078 [41:41<8:28:27,  8.07s/it]07/06/2023 19:26:16 - INFO - __main__ -   Step: 299, LR: 1.91051567239636e-05, Loss: 1.9872896671295166


KeyboardInterrupt: 

# eval



In [16]:
model_name_or_path = 'results/gpt2-Large_human_mix'


cmd = f"""
!cd .. && python -m eval.gsm.run_eval \
    --data_dir data/eval/gsm/ \
    --max_num_examples 200 \
    --model_name_or_path {model_name_or_path} \
    --save_dir {model_name_or_path}/eval/gsm/ \
    --eval_batch_size 20 \
    --n_shot 8
"""
print(cmd)


!cd .. && python -m eval.gsm.run_eval     --data_dir data/eval/gsm/     --max_num_examples 200     --model_name_or_path results/gpt2-Large_human_mix     --save_dir results/gpt2-Large_human_mix/eval/gsm/     --eval_batch_size 20     --n_shot 8



In [22]:
!cd .. && CUDA_LAUNCH_BLOCKING=1 python -m eval.gsm.run_eval     --data_dir data/eval/gsm/     --max_num_examples 200     --model_name_or_path results/gpt2-Large_human_mix     --save_dir results/gpt2-Large_human_mix/eval/gsm/     --eval_batch_size 20     --n_shot 8


Loading data...
Loading model and tokenizer...
Generating Completions:   0%|                           | 0/200 [00:00<?, ?it/s]True
cpu torch.Size([20, 783])
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [108,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [108,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [108,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [108,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [108,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/sr

Error when generating completions for batch:
['Answer the following questions.\n\nQuestion: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nAnswer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. So the answer is 6.\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nAnswer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. So the answer is 5.\n\nQuestion: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nAnswer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. So the answer is 39.\n\nQuestion: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. 

Generating Completions:  10%|█▊                | 20/200 [00:17<02:38,  1.14it/s]True
cpu torch.Size([20, 722])
Traceback (most recent call last):
  File "/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/eval/gsm/run_eval.py", line 144, in <module>
    main(args)
  File "/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/eval/gsm/run_eval.py", line 75, in main
    outputs = generate_completions(
  File "/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/eval/utils.py", line 45, in generate_completions
